# **Use YOLO-cls !**

### (1) 경로 설정

* 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/project4'

### (2) 라이브러리 설치 및 불러오기

* 라이브러리 로딩

In [ ]:
## colab에서 세션 재시작을 요구하는 팝업이 뜨면 재시작 누르세요.
!pip install ultralytics

#### 1) 본인 얼굴 이미지 데이터셋 불러오기

In [ ]:
import os
import zipfile

In [ ]:
data_myFace = os.path.join('/content/drive/MyDrive/project4/my_face_data.zip')
data_myFace

'/content/drive/MyDrive/project4/my_face_data.zip'

In [ ]:
## Colab에 생성할 본인 얼굴 폴더 경로
extract_folder = '/content/drive/MyDrive/project4/Datasets/my_face'

## 위의 경로에 폴더가 없을 때 생성
if not os.path.exists(extract_folder) :
    os.makedirs(extract_folder)

## 위의 경로에 압축을 해제
with zipfile.ZipFile(data_myFace, 'r') as zip_ref :
    file_list = zip_ref.namelist()

    for f in file_list :
        if not f.endswith('/') and f.lower().endswith('.jpg') :
            file_name = os.path.basename(f)

            if not file_name.startswith('._') :
                d_path = os.path.join(extract_folder, file_name)

                with zip_ref.open(f) as source, open(d_path, 'wb') as target :
                    target.write(source.read())

In [ ]:
## 생성된 본인 얼굴 이미지 데이터 폴더 안의 이미지 수
len(os.listdir(extract_folder) )

5000

#### 2) 다른 얼굴 이미지 데이터셋 불러오기

In [ ]:
data_other = path + '/Datasets/Keras/lfw-deepfunneled.zip'
data_other

'/content/drive/MyDrive/project4/Datasets/Keras/lfw-deepfunneled.zip'

In [ ]:
## Colab에 생성할 다른 얼굴 폴더 경로
extract_folder = path+'/Datasets/other_face'

## 위의 경로에 폴더가 없을 때 생성
if not os.path.exists(extract_folder) :
    os.makedirs(extract_folder)

## 위의 경로에 압축을 해제
with zipfile.ZipFile(data_other, 'r') as zip_ref :
    file_list = zip_ref.namelist()

    for f in file_list :
        if not f.endswith('/') and f.lower().endswith('.jpg') :
            file_name = os.path.basename(f)

            if not file_name.startswith('._') :
                d_path = os.path.join(extract_folder, file_name)

                with zip_ref.open(f) as source, open(d_path, 'wb') as target :
                    target.write(source.read())

In [ ]:
## 생성된 다른 사람 얼굴 이미지 데이터 폴더 안의 이미지 수
len(os.listdir(extract_folder) )

13233

In [ ]:
import glob
import random
import shutil
import numpy as np

from keras.utils import load_img, img_to_array
from keras.utils import image_dataset_from_directory

In [ ]:
# 1. 기본 폴더 경로 설정
dataset_root = path+'/Datasets2'
train_data = os.path.join(dataset_root, 'train')
val_data = os.path.join(dataset_root, 'val')
test_data = os.path.join(dataset_root, 'test')

In [ ]:
# 2. YOLO-cls 폴더 구조 생성
for data_folder in [train_data, val_data, test_data]:
    if not os.path.exists(data_folder):
        os.makedirs(data_folder)

# 3. 클래스별 하위 폴더 생성
class_names = ['my', 'other']
for class_name in class_names:
    os.makedirs(os.path.join(train_data, class_name), exist_ok=True)
    os.makedirs(os.path.join(val_data, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_data, class_name), exist_ok=True)

# 4. 이미지 리스트 정리 및 랜덤 셔플
img_list_my = sorted(glob.glob(path+'/Datasets/my_face/*'))
img_list_other = sorted(glob.glob(path+'/Datasets/other_face/*'))

# 데이터 재현성을 위해 랜덤 시드 고정 및 셔플
random.seed(2024)
random.shuffle(img_list_my)
random.shuffle(img_list_other)

# 5. 데이터 분할 (Training: 70%, Validation: 20%, Test: 10%)
def split_data(img_list, train_ratio=0.7, val_ratio=0.2):
    list_len = min(len(img_list), 11000)  # 이미지 최대 개수 제한 (예시로 11000개 사용)
    train_end = int(list_len * train_ratio)
    val_end = int(list_len * (train_ratio + val_ratio))

    return img_list[:train_end], img_list[train_end:val_end], img_list[val_end:list_len]

# 데이터 분할
train_my, val_my, test_my = split_data(img_list_my)
train_other, val_other, test_other = split_data(img_list_other)

# 6. 파일 복사 함수 정의
def copy_files(file_list, dest_folder, class_name):
    for file_path in file_list:
        file_name = os.path.basename(file_path)
        shutil.copy(file_path, os.path.join(dest_folder, class_name, file_name))
        print(f'파일 이동 완료 : {file_name} -> {dest_folder}/{class_name}')

# 7. 데이터셋 폴더로 파일 복사
# 'my' 클래스 데이터 복사
copy_files(train_my, train_data, 'my')
copy_files(val_my, val_data, 'my')
copy_files(test_my, test_data, 'my')

# 'other' 클래스 데이터 복사
copy_files(train_other, train_data, 'other')
copy_files(val_other, val_data, 'other')
copy_files(test_other, test_data, 'other')

print("데이터 분배 완료")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
파일 이동 완료 : Al_Sharpton_0001.jpg -> /content/drive/MyDrive/project4/Datasets2/train/other
파일 이동 완료 : Randy_Ferbey_0001.jpg -> /content/drive/MyDrive/project4/Datasets2/train/other
파일 이동 완료 : Kofi_Annan_0016.jpg -> /content/drive/MyDrive/project4/Datasets2/train/other
파일 이동 완료 : Sargis_Sargsian_0001.jpg -> /content/drive/MyDrive/project4/Datasets2/train/other
파일 이동 완료 : Jamie_Lee_Curtis_0001.jpg -> /content/drive/MyDrive/project4/Datasets2/train/other
파일 이동 완료 : Jean-Patrick_Nazon_0001.jpg -> /content/drive/MyDrive/project4/Datasets2/train/other
파일 이동 완료 : Masaru_Hayami_0001.jpg -> /content/drive/MyDrive/project4/Datasets2/train/other
파일 이동 완료 : Mary_Steenburgen_0003.jpg -> /content/drive/MyDrive/project4/Datasets2/train/other
파일 이동 완료 : Recep_Tayyip_Erdogan_0001.jpg -> /content/drive/MyDrive/project4/Datasets2/train/other
파일 이동 완료 : Abdul_Rahman_0001.jpg -> /content/drive/MyDrive/project4/Datasets2/train/other
파일 이동 완료 : Eliane_Karp_0001.jpg -> /conte

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO, settings

In [ ]:
settings

{'settings_version': '0.0.6',
 'datasets_dir': '/content/datasets',
 'weights_dir': 'weights',
 'runs_dir': 'runs',
 'uuid': '569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb',
 'sync': True,
 'api_key': '',
 'openai_api_key': '',
 'clearml': True,
 'comet': True,
 'dvc': True,
 'hub': True,
 'mlflow': True,
 'neptune': True,
 'raytune': True,
 'tensorboard': True,
 'wandb': False,
 'vscode_msg': True}

In [ ]:
# 1) UltraLytics YOLO-cls 모델 선택 및 불러오기
# 선택 가능한 모델: 'yolov8n-cls.pt', 'yolov8s-cls.pt', 'yolov8m-cls.pt', 'yolov8l-cls.pt', 'yolov8x-cls.pt'
model = YOLO('yolov8n-cls.pt')  # 여기서 n, s, m, l, x 중 하나를 선택

# 2) 데이터 경로 설정 및 모델 학습
# 준비된 데이터셋 경로 지정
dataset_path = '/content/drive/MyDrive/project4/Datasets2'

In [ ]:
# 학습 시작
model.train(data=dataset_path, epochs=10, imgsz=160, batch=32)
# 위에서 'data'는 train과 val 폴더가 포함된 Datasets2 폴더의 경로

Ultralytics 8.3.24 🚀 Python-3.10.12 torch-2.5.0+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/content/drive/MyDrive/project4/Datasets2, epochs=10, time=None, patience=100, batch=32, imgsz=160, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

train: Scanning /content/drive/MyDrive/project4/Datasets2/train... 14356 images, 0 corrupt: 100%|██████████| 14356/14356 [03:01<00:00, 79.20it/s] 


train: New cache created: /content/drive/MyDrive/project4/Datasets2/train.cache


val: Scanning /content/drive/MyDrive/project4/Datasets2/val... 5098 images, 0 corrupt: 100%|██████████| 5098/5098 [00:49<00:00, 102.81it/s]


val: New cache created: /content/drive/MyDrive/project4/Datasets2/val.cache
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 160 train, 160 val
Using 0 dataloader workers
Logging results to runs/classify/train3
Starting training for 10 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...

                   all          1          1



10 epochs completed in 1.809 hours.
Optimizer stripped from runs/classify/train3/weights/last.pt, 3.0MB
Optimizer stripped from runs/classify/train3/weights/best.pt, 3.0MB

Validating runs/classify/train3/weights/best.pt...
Ultralytics 8.3.24 🚀 Python-3.10.12 torch-2.5.0+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv8n-cls summary (fused): 73 layers, 1,437,442 parameters, 0 gradients, 3.3 GFLOPs
train: /content/drive/MyDrive/project4/Datasets2/train... found 14356 images in 2 classes ✅ 
val: /content/drive/MyDrive/project4/Datasets2/val... found 5098 images in 2 classes ✅ 
test: /content/drive/MyDrive/project4/Datasets2/test... found 2622 images in 2 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 80/80 [01:33<00:00,  1.17s/it]


                   all          1          1
Speed: 0.0ms preprocess, 6.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train3


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a15821e58d0>
curves: []
curves_results: []
fitness: 1.0
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 1.0, 'metrics/accuracy_top5': 1.0, 'fitness': 1.0}
save_dir: PosixPath('runs/classify/train3')
speed: {'preprocess': 0.0004413877107808056, 'inference': 6.520390089655352, 'loss': 4.831039470614242e-05, 'postprocess': 3.236282007420189e-05}
task: 'classify'
top1: 1.0
top5: 1.0

In [ ]:
# 예측할 이미지 경로
test_image_path = '/content/drive/MyDrive/project4/Datasets2/test/my'  # 예측할 이미지 파일 경로

# 예측 수행 및 결과 저장
results = model.predict(source=test_image_path, save=True)  # save=True로 하면 결과를 저장함
print(results)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
       [[  0,   3,   1],
        [  0,   4,   2],
        [  2,   6,   1],
        ...,
        [146, 131,  92],
        [145, 130,  91],
        [145, 130,  91]],

       [[  0,   2,   0],
        [  0,   4,   2],
        [  2,   6,   1],
        ...,
        [146, 131,  92],
        [146, 131,  92],
        [145, 130,  91]]], dtype=uint8)
orig_shape: (512, 512)
path: '/content/drive/MyDrive/project4/Datasets2/test/my/474.jpg'
probs: ultralytics.engine.results.Probs object
save_dir: 'runs/classify/train33'
speed: {'preprocess': 8.096933364868164, 'inference': 13.420581817626953, 'postprocess': 0.08893013000488281}, ultralytics.engine.results.Results object with attributes:

boxes: None
keypoints: None
masks: None
names: {0: 'my', 1: 'other'}
obb: None
orig_img: array([[[ 31,  38,  23],
        [ 31,  38,  23],
        [ 30,  37,  24],
        ...,
        [ 85,  82,  61],
        [ 86,  82,  63],
        [ 86,  82,  63]],

       [[ 31,  38,  25],
 

In [ ]:
# 클래스 확률 확인
print(results[5].probs)  # 첫 번째 결과의 클래스 확률 출력

ultralytics.engine.results.Probs object with attributes:

data: tensor([1.0000e+00, 3.6862e-07])
orig_shape: None
shape: torch.Size([2])
top1: 0
top1conf: tensor(1.0000)
top5: [0, 1]
top5conf: tensor([1.0000e+00, 3.6862e-07])


In [ ]:
from google.colab import files
# 4) 학습된 모델 저장 및 다운로드
# 모델을 .pt 형식으로 저장
model_path = '/content/drive/MyDrive/project4/yolov8n-cls-finetuned.pt'
model.save(model_path)

# Colab에서 로컬로 다운로드
files.download(model_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from sklearn.metrics import accuracy_score
import os

# test 데이터셋 경로
test_folder_path = '/content/drive/MyDrive/project4/Datasets2/test'

# 'my' 폴더 예측 수행
results_my = model.predict(source=os.path.join(test_folder_path, 'my'), save=False)
# 'other' 폴더 예측 수행
results_other = model.predict(source=os.path.join(test_folder_path, 'other'), save=False)


image 1/500 /content/drive/MyDrive/project4/Datasets2/test/my/0.jpg: 160x160 my 1.00, other 0.00, 10.3ms
image 2/500 /content/drive/MyDrive/project4/Datasets2/test/my/100.jpg: 160x160 my 1.00, other 0.00, 11.3ms
image 3/500 /content/drive/MyDrive/project4/Datasets2/test/my/1018.jpg: 160x160 my 1.00, other 0.00, 10.4ms
image 4/500 /content/drive/MyDrive/project4/Datasets2/test/my/1029.jpg: 160x160 my 1.00, other 0.00, 10.1ms
image 5/500 /content/drive/MyDrive/project4/Datasets2/test/my/1046.jpg: 160x160 my 1.00, other 0.00, 9.9ms
image 6/500 /content/drive/MyDrive/project4/Datasets2/test/my/1048.jpg: 160x160 my 1.00, other 0.00, 12.9ms
image 7/500 /content/drive/MyDrive/project4/Datasets2/test/my/1076.jpg: 160x160 my 1.00, other 0.00, 9.8ms
image 8/500 /content/drive/MyDrive/project4/Datasets2/test/my/1095.jpg: 160x160 my 1.00, other 0.00, 10.4ms
image 9/500 /content/drive/MyDrive/project4/Datasets2/test/my/1106.jpg: 160x160 my 1.00, other 0.00, 10.1ms
image 10/500 /content/drive/MyDri

In [ ]:
true_labels = []
predicted_labels = []

# 'my' 폴더의 예측 결과
for result in results_my:
    true_labels.append(0)  # 'my' 라벨을 0으로 설정
    predicted_labels.append(result.probs.top1)  # 예측된 라벨 추가

# 'other' 폴더의 예측 결과
for result in results_other:
    true_labels.append(1)  # 'other' 라벨을 1로 설정
    predicted_labels.append(result.probs.top1)  # 예측된 라벨 추가

In [ ]:
from sklearn.metrics import accuracy_score

# 정확도 계산
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Test Set Accuracy: {accuracy * 100:.2f}%")

Test Set Accuracy: 100.00%
